In [2]:
#imports
import re
import numpy as np
from statistics import median


#data
with open('anna.txt', encoding='utf-8') as f:
    anna = f.read()
with open('sonets.txt', encoding='utf-8') as f:
    sonet = f.read()

anna_sentences = re.split(r'(?:[.]\s*){3}|[.?!]', anna)
sonet_sentences = re.split(r'(?:[.]\s*){3}|[.?!]', sonet)       

FileNotFoundError: [Errno 2] No such file or directory: 'anna.txt'

In [ ]:
punct = re.compile(',|-|:')
vowel = re.compile('у|е|ы|а|о|я|и|ю|э')
letter = re.compile('\w')

def count_feach(var, sent_list):
    out_stats = []
    
    for sent in sent_list:
        sent = punct.sub('', sent).lower()
        
        len_count = len(letter.findall(sent))  #длина предложения в буквах
        count_letter = len(set(letter.findall(sent)))  #число различных букв в предложении
        count_vowels = len(vowel.findall(sent))  #число гласных в предложении
        med_letters = 0  #медиана числа букв в слове
        med_vowels = 0  #медиана числа гласных в слове
        
        sent_vowels = []
        sent_letters = []
        for word in sent.split(' '):
            sent_letters.append(len(letter.findall(sent)))
            sent_vowels.append(len(vowel.findall(sent)))
            
        med_letters = median(sorted(sent_letters))
        med_vowels = median(sorted(sent_vowels))
        
        out_stats.append([len_count, count_letter, count_vowels, med_letters, med_vowels, var])
    return out_stats
        
anna_feach = count_feach(1, anna_sentences)
sonet_feach = count_feach(2, sonet_sentences)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

    
def plot_try(i,j,anna=anna_sentences, sonet=sonet_sentences):
    
    anna = np.array(anna)
    sonet = np.array(sonet)
    plt.figure()
    plt.plot(anna[i,:], anna[j,:], 'o')
    plt.show()
    
    #plt.plot(data[:][i], data[:][j], 'ro')
    #plt.show()

anna = np.array(anna_feach)
sonet = np.array(sonet_feach)
#print(anna)   
plt.figure()
plt.plot(anna[:,0], anna[:,1], 'o')
plt.plot(sonet[:,0], sonet[:,1], 'x')
plt.show()

In [ ]:
plt.figure()
plt.plot(anna[:,1], anna[:,2], 'o')
plt.plot(sonet[:,1], sonet[:,2], 'x')
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from math import ceil
import pandas as pd
from sklearn.metrics import classification_report

cols = ['length', 'letter_count', 'vowels', 'med_letters', 'med_vowels', 'var']
ana = pd.DataFrame.from_records(anna_feach, columns=cols)
snt = pd.DataFrame.from_records(sonet_feach, columns=cols)

train = ana[0:int(ceil(len(ana)*0.7))]
train = train.append(snt[0:int(ceil(len(snt)*0.7))])

train.reset_index(level=0, inplace=True)#(range(0,len(train)))

test = ana[int(ceil(len(ana)*0.7)):len(ana)]
test = test.append(snt[int(ceil(len(snt)*0.7)):len(snt)])
test.reset_index(level=0, inplace=True)

rf = RandomForestClassifier()
#print(train[0:])
col = ['length', 'letter_count', 'vowels', 'med_letters', 'med_vowels']
rf.fit(train[col], train['var'])

print(classification_report(rf.predict(test[col]), test['var']))

Видно, что из-за несбаллансированности выборки, точность определения сонетов маленькая, в следующем примере, я делаю более сбаллансированную выборку - беру одинаковое количество предложений для разных типов текста. 
Общие значения получаются хуже, однако более равномерно предсказываются оба класса

In [ ]:
train = ana[0:int(ceil(len(snt)*0.7))]# + snt[0:int(ceil(len(ana)*0.7))]
train = train.append(snt[0:int(ceil(len(snt)*0.7))])
train.reset_index(level=0, inplace=True)
#print(train.head())
test = ana[int(ceil(len(snt)*0.7)):len(snt)]
test = test.append(snt[int(ceil(len(snt)*0.7)):len(snt)])
test.reset_index(level=0, inplace=True)
#print(test.head())
#print(test.head())

rf = RandomForestClassifier()
#print(train[0:])
col = ['length', 'letter_count', 'vowels', 'med_letters', 'med_vowels']
rf.fit(train[col], train['var'])

print(classification_report(rf.predict(test[col]), test['var']))

In [ ]:
c = 0
#print(test.head())
#print(len(test))
#print(test.loc[])

for i in range(1,len(test)):
    #while c <= 3:
    fp = test.loc[i]
    #print(type(fp))
    if rf.predict(fp.loc[col]) != fp['var']:
        print('new mistake')
        print(anna_sentences[int(i+len(snt)*0.7)])
        print(fp.loc[col][i], fp['var'][i])
        print()
        c += 1
        if c == 3:
            break
#Предположение - классификатор ошибается на коротких предложениях